In [57]:
import sys
import os
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import h5py
import json
import requests
import base64
sys.path.append('../')

pd.options.display.max_rows=99

In [58]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'

In [59]:
##pull in trailhead stops
trailheadRef = pd.DataFrame()
trailheadRef_json = 'https://raw.githubusercontent.com/Tanag3r/trailheadDirect_birds/main/trailheadsRef.json'
response = requests.get(trailheadRef_json)
trailheadRef = pd.DataFrame(response.json())
trailheadRef.set_index('StopName')

##check length of lat and long, each must be at least 4 char

,Route,Address,Latitude,Longitude
StopName,,,,
EastSunsetWay,IssaquahAlps,"661-831 E Sunset Way, Issaquah, WA 98027",47.529635,-122.025119
HighSchool,IssaquahAlps,"Parking lot, The Rainier Trail, Issaquah, WA 9...",47.519345,-122.029801
ChiricoTrail_PooPooPoint,IssaquahAlps,"11400 Issaquah-Hobart Road Southeast, Issaquah...",47.499949,-122.021730
SquakMountain,IssaquahAlps,"13201 Squak Mountain Rd SE, Issaquah, WA 98027",47.481465,-122.053997
MargaretsWay,IssaquahAlps,"190th Ave SE, Issaquah, WA 98027",47.506620,-122.086660
MountSi,MountSi,"King County, WA 98104",47.488966,-122.723044
MountTeneriffe,MountSi,"Mount Teneriffe Rd, North Bend, WA 98045",47.490104,-122.709182
LittleSi,MountSi,"SE Mt Si Rd, North Bend, WA 98045",47.489366,-122.753833


In [60]:
##fetch all hotspots within 4 kilometers of each trailhead
##if the number of values in the stop table expand, switch from loop to .apply() function

trailheadHotspots = []

for StopName in trailheadRef.itertuples():
    ##trailhead_stopName = str(StopName.StopName)
    url = '''https://api.ebird.org/v2/ref/hotspot/geo?lat={}&lng={}&dist=4&fmt=json'''.format(StopName.Latitude,StopName.Longitude)
    ebirdapi_auth_header = {'X-eBirdApiToken': ebird_token}
    ebird_request = requests.get(url,headers=ebirdapi_auth_header)
    ebird_response = pd.DataFrame(ebird_request.json())
    trailheadHotspots.append(ebird_response)

ValueError: DataFrame constructor not properly called!

all_trailheadHotspots = pd.concat(trailheadHotspots)

all_trailheadHotspots.head(20)

all_trailheadHotspots.shape

all_trailheadHotspots.to_csv('trailheadHotspots_2022JAN.csv',sep=',',index=False)